# Assignment 3 Top-Level Code/Notebook
### Training a language model base on Karpathy's minGPT codebase


In [1]:
# The code below is needed for using Google Colab, so un comment this if that is what you're using
""" 
import nltk
nltk.download('punkt')
"""

" \nimport nltk\nnltk.download('punkt')\n"

In [2]:
# The code below is also needed for using Google Colab
# BEFORE executing this, you must place the mingpt folder supplied in the assignment
# your google drive, within the folder "Colab Notebooks"
#
# It mounts and changes into the folder that contains mingpt, which you must upload to google drive
# So un-comment it if you've uploaded mingpt to your google drive, into the  "Colab Notebooks" folder
"""
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab\ Notebooks/
""" 

"\nfrom google.colab import drive\ndrive.mount('/content/drive')\n%cd /content/drive/MyDrive/Colab\\ Notebooks/\n"

In [3]:
import torch 
import numpy as np

from nltk.tokenize import sent_tokenize 

from pathlib import Path 
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from mingpt.bpe import BPETokenizer 
from mingpt.utils import set_seed 
set_seed(1234)

In [4]:
"""
Prepare the dataset to train the Language Model (LM)
This implementation splits the sentences and so doesn't create training 
examples that cross sentences.

This code is set so that it uses one of two possible datasets, which were also used in Assignment 1: 
SmallSimpleCorpus.txt or LargerCorpus.txt

Arguments:
            ds_choice: str. "small" or "large". (i.e. selects which of the two datasets)
            split: str. "train" or "test".
            truncation: int. If -1: no truncation on sentences. Otherwise: truncate to this specific length.
""" 

class LanguageModelingDataset(Dataset):
    
    def __init__(self, ds_choice="small", split="train", truncation=-1):
        
        base_path = "./"
        fn = {"small": "SmallSimpleCorpus.txt", "large": "LargerCorpus.txt"}
        self.ds_choice = ds_choice
        self.truncation = truncation  # int. If -1, then
        text = Path(base_path, fn[ds_choice]).read_text()
        if ds_choice == "large":
            # Remove the newline char in the middle of sentences
            # The "paragraph splitting" newlines appear to be \n\n -- remove the duplications there
            text = text.replace("\n\n", "$$^^$$").replace("\n", " ").replace("$$^^$$", "\n")
        sentences = sent_tokenize(text)

        # Train / test split
        train, val = train_test_split(sentences, test_size=0.2, shuffle=False)
        if split == "train":
            raw_data = train 
        else:
            raw_data = val 

        # Tokenize
        self.tokenizer = BPETokenizer()
        self.data = []  # List of 1-d pytorch tensor
        for sent in raw_data:
            tokenized = self.tokenizer(sent).view(-1)  # pytorch tensor
            if truncation >= 0:
                self.data.append(tokenized[:truncation])
            else:
                self.data.append(tokenized)

        # Count some items
        self.max_sentence_length = np.max([len(d) for d in self.data])

    def __len__(self):
        return len(self.data)

    def get_vocab_size(self):
        """
        We have to set this to the max vocab size (i.e., that decided by the BPE tokenizer), 
        but actually, only a small number of vocab is used, especially for the small text. 
        """
        return 50257

    def __getitem__(self, idx):
        """
        The output should be a tuple x and y, both as pytorch tensors.
        Please refer to the `run()` method in the mingpt/trainer.py script for 
        how the x and y are going to be used.
        """
        x = self.data[idx][:-1]
        y = self.data[idx][1:]
        return (x, y)

    def get_block_size(self):
        """
        block_size is the size at which lines are truncated to ensure they are equal-length.
        """
        return self.max_sentence_length
    
# Instantiate the Training Dataset
# train_dataset = LanguageModelingDataset(ds_choice="small", split="train")  # use this for the short corpus
train_dataset = LanguageModelingDataset(ds_choice="large", split="train", truncation=512) #use this for long

# Instantiate a Validation Dataset (this is only really needed for the fine-tune task, not the LM task)
# val_dataset = LanguageModelingDataset(ds_choice="small", split="validation")
val_dataset = LanguageModelingDataset(ds_choice="large", split="validation", truncation=512)

In [7]:
def lm_collate_fn(batch, device):
    x = [item[0] for item in batch]  # List (len B) of varying lengths
    y = [item[1] for item in batch]  # List (len B) of the same lengths as x
    maxlen = max([len(s) for s in x])

    padded_x, padded_y = [], []
    for sx, sy in zip(x, y):
        padded_x.append(torch.cat([sx, torch.ones(maxlen - len(sx))]))
        padded_y.append(torch.cat([sy, torch.ones(maxlen - len(sy))]))
    return torch.stack(padded_x).long().to(device), torch.stack(padded_y).long().to(device)


In [8]:
# Print out an example of the data - this is processed more once it reaches lm_collate_fn (above)
x,y = train_dataset[5]
print(x, y)
print("X: ",train_dataset.tokenizer.decode(x))
print("Y: ",train_dataset.tokenizer.decode(y))

tensor([26788,   286, 11620,   290,  4898,   373,   287, 19133,   287,   262,
         1903,  1528,   286, 10598,    26,   290,   262, 30962,   286,   311,
         1789,    11, 28630,    11,   290,   617,  3354,   286,  5478,   973,
          262, 37928,    12, 25717,  9875,   563,  7582,   284,  2380,  1988,
           11,   290,   617, 23325, 33831,   326,   340,   318,   991,   287,
          779,   287,   262,  6569, 16690,   286,   262,   938,    12, 13190,
         1499]) tensor([  286, 11620,   290,  4898,   373,   287, 19133,   287,   262,  1903,
         1528,   286, 10598,    26,   290,   262, 30962,   286,   311,  1789,
           11, 28630,    11,   290,   617,  3354,   286,  5478,   973,   262,
        37928,    12, 25717,  9875,   563,  7582,   284,  2380,  1988,    11,
          290,   617, 23325, 33831,   326,   340,   318,   991,   287,   779,
          287,   262,  6569, 16690,   286,   262,   938,    12, 13190,  1499,
           13])
X:  Money of leather and wood wa

In [9]:
from mingpt.model import GPT

model_config = GPT.get_default_config()
model_config.model_type = 'gpt-nano'
model_config.vocab_size = train_dataset.get_vocab_size()
model_config.block_size = train_dataset.get_block_size()
model_config.n_classification_class = 2
model = GPT(model_config)

number of parameters: 2.52M


In [10]:
# Create a Trainer object and set the core hyper-parameters
from mingpt.trainer import Trainer

train_config = Trainer.get_default_config()
train_config.learning_rate = 5e-4 # the model we're using is so small that we can go a bit faster
# train_config.max_iters = 3000  # For small corpus: 3000 iterations is plenty. For large corpus: 100000 iterations is needed
train_config.max_iters = 100000
train_config.num_workers = 0
# train_config.batch_size = 4    # For small corpus, batch size of 4 is fine.  For large corpus use 16
train_config.batch_size = 16    # For small corpus, batch size of 4 is fine.  For large corpus use 16
trainer = Trainer(train_config, model, train_dataset, val_dataset, collate_fn=lm_collate_fn)

running on device cpu


In [13]:
# This function is called at the end of every batch in training
# and is used to report the amount of time per 100 batches, and the loss at that point

def batch_end_callback(trainer):
    if trainer.iter_num % 100 == 0:
        print(f"iter_dt {trainer.iter_dt * 1000:.2f}ms; iter {trainer.iter_num}: train loss {trainer.loss.item():.5f}")
trainer.set_callback('on_batch_end', batch_end_callback)

# Train!
trainer.run()

iter_dt 117.79ms; iter 0: train loss 0.94879


KeyboardInterrupt: 

In [14]:
model.to(trainer.device)
# store the saved model in a file, so can re-use later
modelsavename= "model_filename.pt"  # change the name here to save in a specific file (and restore below)
with open(modelsavename, "wb") as f:
    torch.save(trainer.model.state_dict(), f)

In [15]:
# Use the trained language model to predict a sequence of words following a few words
encoded_prompt = train_dataset.tokenizer("He and I").to(trainer.device)
generated_sequence, probs = trainer.model.modified_generate_1(encoded_prompt, trainer.device, temperature=0.8, max_new_tokens=10)
train_dataset.tokenizer.decode(generated_sequence[0])

'He and I can hold a dog. dog. cat. cat'

In [16]:
torch.round(probs, decimals=3)

tensor([[1.0000, 1.0000, 1.0000, 0.3650, 0.5280, 0.5440, 0.5910, 0.9980, 0.4950,
         0.9410, 0.5670, 0.5420, 0.7700]])

In [17]:
# Another example
encoded_prompt = train_dataset.tokenizer("She rubs").to(trainer.device)
generated_sequence, probs = trainer.model.modified_generate_1(encoded_prompt, trainer.device, temperature=0.6, max_new_tokens=10)
train_dataset.tokenizer.decode(generated_sequence[0])

'She rubs a dog. cat. cat. dog and dog'

In [18]:
torch.round(probs, decimals=3)

tensor([[1.0000, 1.0000, 1.0000, 0.4050, 0.6510, 0.8100, 0.9640, 0.9990, 0.6730,
         0.5090, 0.5240, 0.6770, 1.0000]])

In [20]:
# My example
encoded_prompt = train_dataset.tokenizer("I rub").to(trainer.device)
generated_sequence, probs = trainer.model.modified_generate_1(encoded_prompt, trainer.device, temperature=0.6, max_new_tokens=10)
train_dataset.tokenizer.decode(generated_sequence[0])

'I rub a dog. cat.. cat and dog.'

In [21]:
torch.round(probs, decimals=3)

tensor([[1.0000, 1.0000, 0.5100, 0.5960, 0.5500, 0.8380, 0.9710, 0.9350, 0.7110,
         0.5680, 0.9980, 0.9160]])

In [13]:
# The code below shows how to reload the model from the saved file; is useful things that take long to train
model.load_state_dict(torch.load(modelsavename))

<All keys matched successfully>

In [22]:
# Example showing how the reloaded model still works
encoded_prompt = train_dataset.tokenizer("She rubs").to(trainer.device)
generated_sequence = trainer.model.generate(encoded_prompt, trainer.device, temperature=0.6, max_new_tokens=10)
train_dataset.tokenizer.decode(generated_sequence[0])

'She rubs the dog and cat. cat. cat. dog'

In [11]:
# Use the trained language model to predict a sequence of words following a few words
encoded_prompt = train_dataset.tokenizer("He and I").to(trainer.device)
generated_sequence, idx, probs = trainer.model.modified_generate_2(encoded_prompt, trainer.device, temperature=0.8, max_new_tokens=10)
train_dataset.tokenizer.decode(generated_sequence[0])

'He and I can hold a dog. cat. cat and dog'

In [12]:
word = []
for i, x in enumerate(idx):
    col = []
    for j, y in enumerate(x):
        col.append(train_dataset.tokenizer.decode(idx[i][j].reshape(1)) + f"  {probs[i][j]:.3f}")
    word.append(col)
    
import pandas as pd
result = pd.DataFrame(data=word)
result.T

,0,1,2,3,4,5,6,7,8,9
0,can 0.555,hold 0.679,a 0.531,dog 0.614,. 0.998,cat 0.667,. 0.986,cat 0.645,and 0.676,dog 0.991
1,hold 0.288,rub 0.319,the 0.465,cat 0.386,. 0.002,dog 0.331,and 0.012,dog 0.353,. 0.321,cat 0.005
2,rub 0.152,can 0.001,and 0.004,a 0.000,and 0.000,a 0.000,. 0.002,a 0.001,a 0.001,can 0.001
3,holds 0.003,the 0.000,hold 0.000,the 0.000,rub 0.000,the 0.000,a 0.000,the 0.001,the 0.001,rub 0.001
4,and 0.000,a 0.000,cat 0.000,and 0.000,dog 0.000,and 0.000,the 0.000,and 0.001,can 0.000,holds 0.001
5,dog 0.000,dog 0.000,holds 0.000,rub 0.000,cat 0.000,. 0.000,cat 0.000,. 0.000,. 0.000,and 0.000


In [13]:
train_dataset.tokenizer.decode(generated_sequence[0][0].reshape(1))

'He'

In [16]:
# Another example
encoded_prompt = train_dataset.tokenizer("She rubs").to(trainer.device)
generated_sequence, idx, probs = trainer.model.modified_generate_2(encoded_prompt, trainer.device, temperature=0.6, max_new_tokens=10)
train_dataset.tokenizer.decode(generated_sequence[0])

'She rubs a cat and dog. cat. cat. dog'

In [17]:
word = []
for i, x in enumerate(idx):
    col = []
    for j, y in enumerate(x):
        col.append(train_dataset.tokenizer.decode(idx[i][j].reshape(1)) + f"  {probs[i][j]:.3f}")
    word.append(col)
    
import pandas as pd
result = pd.DataFrame(data=word)
result.T

,0,1,2,3,4,5,6,7,8,9
0,a 0.493,cat 0.569,and 0.679,dog 0.998,. 0.999,cat 0.508,. 0.998,cat 0.724,. 0.998,dog 0.776
1,the 0.387,dog 0.431,. 0.321,cat 0.002,. 0.001,dog 0.489,and 0.001,dog 0.275,. 0.001,cat 0.224
2,and 0.120,and 0.000,a 0.000,a 0.000,and 0.000,a 0.001,. 0.000,and 0.001,rub 0.000,a 0.000
3,. 0.000,. 0.000,the 0.000,the 0.000,cat 0.000,and 0.001,rub 0.000,a 0.000,I 0.000,the 0.000
4,holds 0.000,holds 0.000,. 0.000,rub 0.000,dog 0.000,the 0.001,can 0.000,the 0.000,and 0.000,and 0.000
5,rub 0.000,rub 0.000,can 0.000,and 0.000,rub 0.000,. 0.000,holds 0.000,. 0.000,hold 0.000,holds 0.000


In [97]:
# Create and config model
model_config = GPT.get_default_config()
model_config.model_type = 'gpt-nano'
model_config.vocab_size = train_dataset.get_vocab_size()
model_config.block_size = train_dataset.get_block_size()
model_config.n_classification_class = 2
model = GPT(model_config)

# Load state dict
modelsavename= "model_large100K.pt"
model.load_state_dict(torch.load(modelsavename))

# Create a Trainer object and set the core hyper-parameters
train_config = Trainer.get_default_config()
trainer = Trainer(train_config, model, train_dataset, val_dataset, collate_fn=lm_collate_fn)

# Test the results
encoded_prompt = train_dataset.tokenizer("She flips").to(trainer.device)
generated_sequence = trainer.model.generate(encoded_prompt, trainer.device, temperature=0.6, max_new_tokens=10)

result = train_dataset.tokenizer.decode(generated_sequence[0])

print(result)

number of parameters: 2.52M
running on device cpu
She flips in the coins is always room. were
 


In [129]:
# Test the results
encoded_prompt = train_dataset.tokenizer("I like").to(trainer.device)
generated_sequence = trainer.model.generate(encoded_prompt, trainer.device, temperature=0.6, max_new_tokens=10)

result = train_dataset.tokenizer.decode(generated_sequence[0])

print(result)

I like that time he was passed by Mr.
 
